visualize pcd

In [ ]:
import open3d as o3d
import numpy as np
import os
import json
from tqdm import tqdm
from scipy.spatial.transform import Rotation as R
import time
def quat2rot(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1 - 2*qy**2 - 2*qz**2,   2*qx*qy - 2*qz*qw,     2*qx*qz + 2*qy*qw],
        [2*qx*qy + 2*qz*qw,       1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
        [2*qx*qz - 2*qy*qw,       2*qy*qz + 2*qx*qw,     1 - 2*qx**2 - 2*qy**2]
    ])

def get_pose(state):
    p = state['kinematics_estimated']['position']
    o = state['kinematics_estimated']['orientation']
    pos = np.array([p['x_val'], p['y_val'], p['z_val']], dtype=float)
    rot = quat2rot(o)
    return pos, rot

# KITTI color palette
label_to_color = {
    0: (0.5, 0.5, 0.5),
    1: (0.0, 1.0, 0.0),
    2: (1.0, 0.0, 0.0),
}

# ### CORRECT conversion from your PLY frame (X→right, Y→forward) 
# ###      → KITTI (X→forward, Y→left) is a –90° about Z, not –180°.
R_kitti = R.from_euler('z', -90, degrees=True).as_matrix()

output_frame_dir = "rendered_frames"
# os.makedirs(output_frame_dir, exist_ok=True)

for file_num in tqdm(range(8), desc="Tracks"):
    track_out = os.path.join(output_frame_dir, str(file_num))
    os.makedirs(track_out, exist_ok=True)

    data_dir = f'../data/{file_num}'
    ply_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.ply')],
                       key=lambda x: int(x[:-4]))

    # one window per track
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=1280, height=720, visible=False)

    ctr = vis.get_view_control()
    ctr.set_zoom(0.7)
    for ply_name in tqdm(ply_files, desc=f"Frame in track {file_num}"):
        idx = ply_name[:-4]

        # --- load
        pcd = o3d.io.read_point_cloud(os.path.join(data_dir, ply_name))
        labels = np.load(os.path.join(data_dir, f"{idx}_labels.npy")).astype(int).reshape(-1)
        with open(os.path.join(data_dir, f"{idx}_car_state.json")) as f:
            car_state = json.load(f)

        # --- colorize
        cols = np.array([label_to_color.get(l, label_to_color[0]) for l in labels])
        pcd.colors = o3d.utility.Vector3dVector(cols)


        # --- get pose & flatten to yaw only
        pos, rot_full = get_pose(car_state)
        yaw = np.arctan2(rot_full[1, 0], rot_full[0, 0])
        rot_flat = R.from_euler('z', yaw, degrees=False).as_matrix()

        # --- transform into world with no roll/pitch
        T = np.eye(4)
        T[:3, :3] = rot_flat @ R_kitti
        T[:3, 3] = pos
        # pcd.transform(T)

        # --- chase‐cam in flattened frame
        offset_local = np.array([0, 3, -8.0])
        cam_pos = pos + rot_flat @ (R_kitti @ offset_local)
        lookat = pcd.get_center() + np.array([0, 0, 0.2])

        # --- render
        vis.clear_geometries()
        vis.add_geometry(pcd)
        
        ctr.set_lookat( lookat )
        front = (lookat - cam_pos)
        front /= np.linalg.norm(front)
        ctr.set_front( front.tolist() )
        ctr.set_up([0, 0, 1])
        

        vis.poll_events()
        vis.update_renderer()
        vis.run()
        # time.sleep(0.1)
        img_path = os.path.join(track_out, f"{idx}.png")
        vis.capture_screen_image(img_path)

    vis.destroy_window()


2

In [ ]:
import open3d as o3d
import numpy as np
import os, re
from tqdm import tqdm
import json
def quat2rot(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1 - 2*qy**2 - 2*qz**2,   2*qx*qy - 2*qz*qw,     2*qx*qz + 2*qy*qw],
        [2*qx*qy + 2*qz*qw,       1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
        [2*qx*qz - 2*qy*qw,       2*qy*qz + 2*qx*qw,     1 - 2*qx**2 - 2*qy**2]
    ])

def get_pose(state):
    p = state['kinematics_estimated']['position']
    o = state['kinematics_estimated']['orientation']
    pos = np.array([p['x_val'], p['y_val'], p['z_val']], dtype=float)
    return pos, quat2rot(o)
output_frame_dir = "rendered_frames"
os.makedirs(output_frame_dir, exist_ok=True)

for file_num in tqdm(range(1,2)):  # adjust end as needed
    file_output_dir = os.path.join(output_frame_dir, str(file_num))
    os.makedirs(file_output_dir, exist_ok=True)

    # find all .ply files in ../data/<file_num> and sort by frame index
    names = os.listdir(f'../data/{file_num}')
    pattern_ply = re.compile(r'^[^\.]+\.ply$')
    names_ply = sorted(
        [n for n in names if pattern_ply.match(n)],
        key=lambda x: int(os.path.splitext(x)[0])
    )

    frame_id = 0
    for ply_name in names_ply:
        idx = os.path.splitext(ply_name)[0]

        # — Load point cloud —
        pcd = o3d.io.read_point_cloud(f'../data/{file_num}/{ply_name}')

        # — Load precomputed labels —
        labels = np.load(f'../data/{file_num}/{idx}_labels.npy')\
                   .reshape(-1).astype(int)
        car = json.load(open(f'../data/{file_num}/{idx}_car_state.json'))

        # — Assign colors by label —
        label_to_color = {
            0: (0.5, 0.5, 0.5),  # unknown/background
            1: (0.0, 1.0, 0.0),  # class 1 in green
            2: (1.0, 0.0, 0.0),  # class 2 in red
            # add more mappings if needed
        }
        colors = np.array([label_to_color.get(l, (0.5, 0.5, 0.5)) for l in labels])
        pcd.colors = o3d.utility.Vector3dVector(colors)

        # — Off‐screen render to PNG —
        vis = o3d.visualization.Visualizer()
        vis.create_window(
            window_name=f"SemKITTI-style: Track {file_num} Frame {idx}", visible=True
        )
        vis.add_geometry(pcd)
        ctr = vis.get_view_control()
        
        pos, R = get_pose(car)

        front = R[:, 0]  # or R[:, 1], depending on which column matches car facing
        up = R[:, 2]     # assuming Z-up convention
        lookat = pos + front

        ctr.set_front(-front / np.linalg.norm(front))
        ctr.set_up(up / np.linalg.norm(up))
        ctr.set_lookat(lookat)
        ctr.set_zoom(0.01)
            # camera up is +Z
        vis.update_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()
        vis.run()
        out_path = os.path.join(file_output_dir, f"frame_{frame_id:04d}.png")
        # vis.capture_screen_image(out_path)
        vis.destroy_window()

        print(f"Saved frame {frame_id} to {out_path}")
        frame_id += 1